<a href="https://colab.research.google.com/github/berkindurmus/SLIViT/blob/main/SLIViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
import torch
import torchvision.models as tmodels
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import nn

In [ ]:
def load_backbone(model_name):
    logging.info("Loading ImageNet Model")
    model = tmodels.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1')
    return torch.nn.Sequential(*list(model.children())[:-2])
def pair(t):
    return t if isinstance(t, tuple) else (t, t)
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x


class SLIViT(nn.Module):
    def __init__(self, *, backbone, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool='cls',
                 channels=3, dim_head=64, dropout=0., emb_dropout=0.):
        super().__init__()
        self.backbone = backbone
        self.channels=channels
        self.image_size=image_size
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        patch_height=patch_height*8
        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'
        num_patches = (image_height // patch_height) * (image_width // patch_width) *channels
        patch_dim = patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'
        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (c h w) (p1 p2)', p1=patch_height, p2=patch_width),
            nn.Linear(patch_dim, dim),
        )
        tmp_pos_emb = torch.zeros((1, dim))
        orderOSlcs = torch.arange(num_patches)+1
        for i in range(num_patches):tmp_pos_emb=torch.concat([tmp_pos_emb,torch.ones((1,dim))*orderOSlcs[i]],axis=0)
        self.pos_embedding = nn.Parameter(tmp_pos_emb.reshape((1, tmp_pos_emb.shape[0], tmp_pos_emb.shape[1])))#.cuda()
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)
        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
        self.pool = pool
        self.to_latent = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )
        self.act = nn.Sigmoid()

    def forward(self, img):
        #Backone
        img = self.backbone(img).reshape((img.shape[0], self.image_size[0], self.channels, self.image_size[1]))
        img = img.permute(0, 2, 1, 3)
        #Feature Maps to Patches
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape
        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        #Add Slice Order as Positional Embedding for Encoding 3D Spatial Info
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)
        x = self.transformer(x)
        x = x.mean(dim=1) if self.pool == 'mean' else x[:, 0]
        x = self.to_latent(x)
        x = self.mlp_head(x)
        x=torch.squeeze(x)
        return x

###Load Feature Extractor (Backbone)

In [ ]:
backbone = load_backbone('ImageNet')

###Create SLIViT Model


In [ ]:

m_SLIViT=SLIViT(backbone=backbone, image_size=(512, 64), patch_size=64, num_classes=1, dim=256, depth=5, heads=19,
    mlp_dim=512, channels=19, dropout=0.3, emb_dropout=0.4)

####Generate Random Data

In [ ]:
nObatch=4
nOslices=19
nOch=3
image_size=(256,256)
rand_long_images=torch.rand(nObatch,nOch,nOslices*image_size[0], image_size[1])

####Make Inference Using Generated Data

In [ ]:
scores=m_SLIViT(rand_long_images)

In [ ]:
print(scores.detach().numpy())

[-2.6059151e-04  3.6952800e-01  1.1123219e+00  1.0020475e+00]
